In [1]:
import os
import sys
base_dir = os.path.join("/home/jovyan/work/Corona/COVID-19")
sys.path.append(base_dir)
import model as md
import pandas as pd

In [2]:
#NewsExtractor(2)
def createJson(date, day, json=False):
    news_reader = md.NewsReader(data_dir="data", month=date, date="{}-{}".format(date, day), string_filter=True)
    repartition = news_reader.repartition[
        (news_reader.repartition["h1"]=="Az aktív fertőzöttek megyei eloszlása")
        & (news_reader.repartition["date"].dt.day == int(day))
    ]["body"].unique()
    date_of_info = news_reader.repartition[
        (news_reader.repartition["h1"]=="Az aktív fertőzöttek megyei eloszlása")
        & (news_reader.repartition["date"].dt.day == int(day))
    ]["date"].unique()
    print(date_of_info)
    if len(repartition)>0:
        table = "Aktív fertőzöttek száma{}".format(repartition[0].split("Aktív fertőzöttek száma")[1]).split("\n\n\n\n\n")
        df = pd.DataFrame(table, columns=["full"])
        df["full"] = df["full"].str.split("\n\n\n")
        df["len"] = df["full"].apply(lambda cell: len(cell))
        df = df[df["len"]==3].copy()
        df['region'], df['active'], df['death'] = zip(*df["full"])
        df['date'] = "{}-{}".format(date, day)
        df[['date', 'region', 'active', 'death']].to_json('repartition/{}-{}.json'.format(date, day), orient='records')
        return df[['date', 'region', 'active', 'death']]

In [7]:
createJson("2021-08", "05", True)

INFO:root:2021-08-05 09:23:00
INFO:root:https://koronavirus.gov.hu/cikkek/5-millio-637-ezer-beoltott-72-az-uj-fertozott-nincs-ujabb-elhunyt
INFO:root:
A beoltottak száma 5 636 894 fő, közülük 5 475 471 már a második oltását is megkapta. 72 új fertőzöttet igazoltak, ezzel a járvány kezdete óta összesen 809 803 főre nőtt a beazonosított fertőzöttek száma. Nincs újabb elhunyt, így az elhunytak száma változatlanul 30 032 fő. A gyógyultak száma folyamatosan nő, jelenleg 750 955 fő, az aktív fertőzöttek száma pedig 28 816 főre csökkent. 79 koronavírusos beteget ápolnak kórházban, közülük 11-en vannak lélegeztetőgépen. A delta vírusmutáns több európai országban és már hazánkban is terjed, ezért kérjük, hogy aki még nem oltatta be magát, tegye meg. A még be nem oltott időseket különösen kérjük, hogy vegyék fel az oltást! Európában elsőként Magyarország teszi lehetővé a koronavírus elleni harmadik védőoltás felvételét,  amelyre az eeszt.gov.hu honlapon már lehet időpontot foglalni. Eddig 35 ezr

In [4]:
def extractJson(date, day, json=False):
    my_object = md.NewsReader(data_dir="data", month=date, date="{}-{}".format(date, day), string_filter=True)
    my_object.repartition["selected"] = my_object.repartition.apply(
        lambda row: my_object.show(row["date"], row["url"], row["body"], date, True),
        axis=1
    )
    return my_object

In [7]:
my_object = extractJson("2021-03", "30", True)
my_object.repartition

INFO:root:2021-03-30 12:11:00
INFO:root:https://koronavirus.gov.hu/cikkek/ezekben-az-orakban-atlepjuk-2-millio-beoltottat
INFO:root:
Folyamatosan zajlik az oltás, ezekben az órákban átlépjük a 2 millió beoltottat. A brit vírusmutáns nagyon gyorsan terjed, jelentősen nő az aktív fertőzöttek és a kórházi betegek száma, ezért a korlátozások továbbra is érvényben vannak. Kérjük, hogy tartsák be az óvintézkedéseket! 4609 az új fertőzöttek száma, ezzel a járvány kezdete óta összesen 645 733 főre nőtt a beazonosított fertőzöttek száma. Elhunyt 274 többségében idős, krónikus beteg, így az elhunytak száma 20 435 főre emelkedett. A gyógyultak száma folyamatosan nő, jelenleg 402 964 fő, de az aktív fertőzöttek száma is már 222 334 főre emelkedett.12 553 koronavírusos beteget ápolnak kórházban, közülük 1529-en vannak lélegeztetőgépen. Az oltási program rendben, ütemezetten zajlik, Magyarország a második az uniós oltási ranglistán. A magyar lakosság átoltottsága elérte a 19,7%-ot szemben a 10,6%-os

INFO:root:2021-03-01 09:08:00
INFO:root:https://koronavirus.gov.hu/cikkek/4326-fovel-emelkedett-beazonositott-fertozottek-szama-es-elhunyt-84-beteg
INFO:root:
4 326  újabb magyar állampolgárnál mutatták ki az új koronavírus-fertőzést (COVID-19), ezzel 432 925 főre nőtt a hazánkban beazonosított fertőzöttek száma. Elhunyt 84 többségében idős, krónikus beteg, így az elhunytak száma 15 058 főre emelkedett. A gyógyultak száma folyamatosan nő, jelenleg 322 956 fő, de az aktív fertőzöttek száma is 94 911 főre emelkedett. 5 679 koronavírusos beteget ápolnak kórházban, közülük 537-en vannak lélegeztetőgépen. Eddig 685 247 fő kapott oltást, közülük 251 691 fő már a második oltását is megkapta. A háziorvosok továbbra is végzik az oltást a kínai Sinopharm-vakcinával, minden háziorvos átlagosan 50-55 fő regisztrált időset tud beoltani, emellett folytatják a 60 év alatti krónikus betegek oltását az AstraZeneca-vakcinával, amellyel praxisonként 10 fő beoltására van lehetőség. A kórházi oltópontokon 

INFO:root:2021-03-01 09:34:00
INFO:root:https://koronavirus.gov.hu/cikkek/merkely-jelenleg-cirkulalo-virus-sulyos-megbetegedeseket-okozhat
INFO:root:
A járvány harmadik hullámában cirkuláló vírus súlyos megbetegedéseket okozhat - hívta fel a figyelmet a Semmelweis Egyetem rektora hétfőn a Kossuth rádió Jó reggelt, Magyarország! című műsorában.


Merkely Béla arról beszélt, hogy az egészségügy érzi a harmadik hullám hatását: a Covid-ellátó részlegek és az intenzív osztályok telítődnek, a lélegeztetőgépen lévők száma is folyamatosan nő.
    Közlése szerint ennek oka lehet több új vírusmutáció megjelenése, amelyek elsősorban a középkorúak és a fiatalok között "talán még jobban terjednek", gyakran súlyos tüdőgyulladást okozva.
    Az ellátórendszer terhelődéséről úgy fogalmazott: "azért van még tartalék", majd azt hangsúlyozta, hogy az egészségügyi dolgozók vakcinációja miatt az ő munkaképességük és biztonságuk sokkal magasabb fokú, mint a járvány előző hullámaiban.
    Az oltásokra térve 

,date,url,h1,body,selected
0,2021-03-01 07:00:00,https://koronavirus.gov.hu/cikkek/mit-kell-tud...,Mit kell tudni a regisztrált legidősebbek és a...,\nFebruár első hetén elkezdődött a regisztrált...,[]
0,2021-03-01 09:08:00,https://koronavirus.gov.hu/cikkek/4326-fovel-e...,4326 fővel emelkedett a beazonosított fertőzöt...,\n4 326 újabb magyar állampolgárnál mutatták ...,[\n4 326 újabb magyar állampolgárnál mutatták...
0,2021-03-01 09:15:00,https://koronavirus.gov.hu/cikkek/fertozottek-...,"A fertőzöttek száma a világon 114 millió, a ha...",\nA világon 114 067 979 ember fertőződött meg ...,[]
0,2021-03-01 09:34:00,https://koronavirus.gov.hu/cikkek/merkely-jele...,Merkely: a jelenleg cirkuláló vírus súlyos meg...,\nA járvány harmadik hullámában cirkuláló víru...,[\nA járvány harmadik hullámában cirkuláló vír...
0,2021-03-01 09:58:00,https://koronavirus.gov.hu/cikkek/novak-jelzal...,Novák: a jelzálogtartozás csökkentésével párhu...,\nA jelzálogtartozás csökkentésével párhuzamos...,[]
...,...,...,...,...,...
0,2021-03-30 14:36:00,https://koronavirus.gov.hu/cikkek/orszagos-tis...,Országos tisztifőorvos: Magyarország oltónemzet,"\n""Magyarország oltónemzet"", rövid időn belül ...",[]
0,2021-03-30 14:48:00,https://koronavirus.gov.hu/cikkek/atlepte-2-mi...,Átlépte a 2 milliót a beoltottak száma,\nÁtlépte a 2 milliót a beoltottak száma. Magy...,[]
0,2021-03-30 15:17:00,https://koronavirus.gov.hu/cikkek/gyorgy-istva...,György István: ezekben az órákban lépi át a be...,\nEzekben az órákban lépi át a kétmilliót a be...,[]
0,2021-03-30 17:36:00,https://koronavirus.gov.hu/cikkek/az-oltast-ke...,"Az oltást kérő várandósok regisztráljanak, maj...",\nAz Országos oltási munkacsoport és a háziorv...,[]
